<a href="https://colab.research.google.com/github/PaulYYLin/MINST_KaggleComp/blob/main/MINST_kaggle_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Kaggle Connection

In [1]:
!pip install kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json



Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes


In [ ]:
!kaggle competitions list

In [2]:
!kaggle competitions download "digit-recognizer"

 65% 10.0M/15.3M [00:00<00:00, 101MB/s]
100% 15.3M/15.3M [00:00<00:00, 126MB/s]


In [3]:
!unzip digit-recognizer.zip

Archive:  digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
import pandas as pd

### Data Description
Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

In [5]:
train = pd.read_csv('train.csv')

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

In [ ]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data Preparation

In [6]:
import tensorflow as tf
import numpy as np

In [7]:
x = train.iloc[:,1:]
y = train.iloc[:,0]


data = tf.data.Dataset.from_tensor_slices((x,y))


In [8]:
print(x.shape, y.shape)

(42000, 784) (42000,)


In [9]:
# Data casting
def feature_scale(x,y):
  x = tf.cast(x, dtype=tf.float32)
  y = tf.cast(y, dtype = tf.int32)
  return x,y

# Label to one-hot
y = tf.keras.utils.to_categorical(
    y, num_classes=10)

# map: 將每一筆資料帶入 function (cast), shuffle: 重混資料, 設定batch_size
data = data.map(feature_scale).shuffle(10000).batch(128)

In [10]:
data_iter = iter(data)
sample = next(data_iter)

print(sample[0].shape, sample[1].shape)

(128, 784) (128,)


## Model Compile

#### DNN

In [7]:
#定義Model

model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.build(input_shape=[1,28*28])

In [ ]:
tf.keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

tf.keras.losses https://www.tensorflow.org/api_docs/python/tf/keras/losses

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = ['crossentropy'],
)
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (1, 256)                  200960    
                                                                 
 dense_55 (Dense)            (1, 128)                  32896     
                                                                 
 dense_56 (Dense)            (1, 64)                   8256      
                                                                 
 dense_57 (Dense)            (1, 32)                   2080      
                                                                 
 dense_58 (Dense)            (1, 16)                   528       
                                                                 
 dense_59 (Dense)            (1, 10)                   170       
                                                                 
Total params: 244,890
Trainable params: 244,890
Non-tr

#### CNN

- VGG Method

In [11]:
# Convolution & Pooling
VGG_layers = [

# Stack1
tf.keras.layers.Conv2D(64, kernel_size=3, strides=1,activation=tf.nn.relu, padding='same'),
tf.keras.layers.Conv2D(64, kernel_size=3, strides=1,activation=tf.nn.relu, padding='same'),
tf.keras.layers.MaxPool2D(2,strides=2),

#stack2
tf.keras.layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
tf.keras.layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),

#stack3
tf.keras.layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
tf.keras.layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
tf.keras.layers.Conv2D(256,kernel_size=[1,1],padding='same',activation=tf.nn.relu),
tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=7,padding='same'),
]
# FC & Flat
fc_layers=[
tf.keras.layers.Dense(128,activation=tf.nn.relu),
tf.keras.layers.Dense(64,activation=tf.nn.relu),
tf.keras.layers.Dense(32,activation=tf.nn.relu),
tf.keras.layers.Dense(16,activation=tf.nn.relu),
tf.keras.layers.Dense(10,activation=tf.nn.softmax)
]

In [12]:
vgg = tf.keras.Sequential(VGG_layers)
vgg.build(input_shape=[None,28,28,1])

fc = tf.keras.Sequential(fc_layers)
fc.build(input_shape=[None,256])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [117]:
vgg.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_71 (Conv2D)          (None, 28, 28, 64)        640       
                                                                 
 conv2d_72 (Conv2D)          (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_73 (Conv2D)          (None, 14, 14, 128)       73856     
                                                                 
 conv2d_74 (Conv2D)          (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 7, 7, 128)        0         
 g2D)                                                

In [118]:
fc.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 128)               32896     
                                                                 
 dense_45 (Dense)            (None, 64)                8256      
                                                                 
 dense_46 (Dense)            (None, 32)                2080      
                                                                 
 dense_47 (Dense)            (None, 16)                528       
                                                                 
 dense_48 (Dense)            (None, 10)                170       
                                                                 
Total params: 43,930
Trainable params: 43,930
Non-trainable params: 0
_________________________________________________________________


## Model Fit / Evaluate

In [13]:
# history = fc.fit(tf.reshape(vgg.(x), [-1,256]), y, epochs=10, validation_split = 0.1)

variables = vgg.variables + fc.variables
for i in range(5):
  for step,(x,y) in enumerate(data):
    # print(x.shape, y.shape)
    with tf.GradientTape() as tape:
      logits = vgg(tf.reshape(x,[-1,28,28]))
      # print(logits.shape)
      logits = tf.reshape(logits,[-1,256])
      # print(logits.shape)
      logits = fc(logits)
      y_one_hot = tf.one_hot(y,depth=10)
      loss = tf.losses.categorical_crossentropy(y_one_hot,logits)
      loss = tf.reduce_mean(loss)
    grads = tape.gradient(loss,variables)
    optimizer.apply_gradients(zip(grads,variables))


In [34]:
vgg.save('VGG_Ep10_test')

## Model Predict

In [ ]:
model.load('Mymodel')

In [62]:
test = pd.read_csv('test.csv')
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
test_x = tf.reshape(test.values,[-1,28,28])

logits = vgg.predict(test_x)
logits = tf.reshape(logits,[-1,256])
logits = fc.predict(logits)
final_hat = [ np.argmax(x) for x in logits]


875/875 [==============================] - 1s 1ms/step


In [64]:
result = pd.read_csv('sample_submission.csv')
result['Label'] = final_hat
result.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [65]:
result.to_csv('vgg_hat.csv', index=False)

Model H1 predict result: Acc 0.9751

Model vgg predict result: Acc 0.97442